In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# Wrangle


In [11]:
constructs=pd.read_csv('data/construct_measures.csv', index_col='variable')

In [16]:
# Pick target
target = 'USE1'
print(target,'-', constructs.loc[target].statement)

USE1 - I use Wikipedia to develop my teaching materials


In [32]:
def wrangle(df):
    df = df.copy()
    
    # Remap target to binary and drop neutral (3.0) values and null values
    df[target] = df[target].map({5:'Yes', 4: 'Yes', 2: 'No', 1: 'No'})
    df.dropna(subset=[target])

    return df

# 
df = wrangle(pd.read_csv('data/wiki4HE_train.csv',index_col=0))

In [33]:
df.sample(10)

,AGE,GENDER,DOMAIN,PHD,YEARSEXP,UNIVERSITY,UOC_POSITION,OTHER_POSITION,OTHERSTATUS,USERWIKI,...,BI2,INC1,INC2,INC3,INC4,EXP1,EXP2,EXP3,EXP4,EXP5
365,34,0,6.0,0,3.0,1,6.0,2.0,NaN,1.0,...,5.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,3.0,1.0
207,44,0,4.0,0,1.0,1,6.0,2.0,NaN,1.0,...,4.0,4.0,3.0,3.0,3.0,5.0,5.0,5.0,4.0,4.0
566,42,1,6.0,0,6.0,1,6.0,1.0,7.0,0.0,...,3.0,4.0,4.0,2.0,3.0,3.0,3.0,2.0,1.0,2.0
674,45,1,1.0,0,3.0,1,6.0,2.0,NaN,0.0,...,2.0,5.0,5.0,5.0,5.0,2.0,2.0,2.0,2.0,4.0
300,40,0,6.0,1,18.0,1,6.0,1.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN
432,58,0,4.0,1,30.0,1,6.0,1.0,2.0,0.0,...,2.0,2.0,4.0,4.0,4.0,2.0,3.0,3.0,1.0,1.0
334,41,0,2.0,0,3.0,1,6.0,2.0,NaN,0.0,...,2.0,3.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0
531,43,1,3.0,0,15.0,1,6.0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,4.0,4.0,5.0,1.0,5.0
24,38,0,5.0,1,14.0,1,2.0,NaN,NaN,1.0,...,3.0,5.0,5.0,3.0,2.0,3.0,4.0,4.0,2.0,2.0
28,35,0,6.0,0,10.0,1,3.0,NaN,NaN,0.0,...,3.0,4.0,4.0,3.0,4.0,3.0,3.0,4.0,3.0,3.0


# Split Data

# Baseline Metrics

# Build Base Model

# Check Metrics

# Tune parameters

# Communicate Results